In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_raw_1 = pd.read_csv('goodreads/books_d.csv')
data_raw_2 = pd.read_csv('goodreads/books_q.csv')
data_raw_3 = pd.read_csv('goodreads/books_m.csv')

In [4]:
data = pd.concat([data_raw_1, data_raw_2, data_raw_3])
len(data)

456032

In [5]:
avg_rating = data['Rate']
avg_rating.head()

0    4.57
1    4.50
2    4.47
3    4.43
4    4.57
Name: Rate, dtype: float64

In [6]:
clean_v1 = data[~data.GenreLink.isnull()]

len(clean_v1)

281522

In [7]:
clean_v2 = clean_v1[clean_v1["Raters"] > 100]
len(clean_v2)

174894

In [9]:
clean_v3 = clean_v2[clean_v2["Pages"] > 0]
len(clean_v3)

152079

In [10]:
clean_v4 = clean_v3.drop_duplicates(subset="GenreLink")
len(clean_v4)

97618

In [12]:
conditions = [
    (clean_v4['Rate'] < 3),
    (clean_v4['Rate'] >= 3) & (clean_v4['Rate'] < 3.8),
    (clean_v4['Rate'] >= 3.8) 
    ]

values = [0, 1, 2]

# create a new column and use np.select to assign values to it using our lists as arguments
clean_v4['review'] = np.select(conditions, values)

clean_v4.head()

C:\Users\Dell\Miniconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,BookID,Title,Author,Rate,Raters,Reviewers,Pages,PublishYear,GenreLink,Series,review
0,1,Harry Potter and the Half-Blood Prince,J.K. Rowling,4.57,2469197,40043,652.0,2005,/work/shelves/41335427,Harry Potter,2
1,2,Harry Potter and the Order of the Phoenix,"J.K. Rowling, Mary GrandPré",4.50,2541611,43067,870.0,2003,/work/shelves/2809203,Harry Potter,2
2,3,Harry Potter and the Sorcerer's Stone,"J.K. Rowling, Mary GrandPré",4.47,7145889,113905,309.0,1997,/work/shelves/4640799,Harry Potter,2
3,4,Harry Potter and the Chamber of Secrets,J.K. Rowling,4.43,2766218,53651,352.0,1998,/work/shelves/6231171,Harry Potter,2
4,5,Harry Potter and the Prisoner of Azkaban,"J.K. Rowling, Mary GrandPré",4.57,2849671,56337,435.0,1999,/work/shelves/2402163,Harry Potter,2


In [13]:
clean_v4.to_csv('goodreads/actual_data.csv', index=False)

In [18]:
clean_v4[~clean_v4['PublishYear'].str.isdigit()]

,BookID,Title,Author,Rate,Raters,Reviewers,Pages,PublishYear,GenreLink,Series,review
28532,29906,The Collected Dialogues,"Plato, Edith Hamilton, Huntington Cairns",4.43,706,31,1776.0,uary,/work/shelves/46180345,NaN,2
133358,441945,Dhammapada: Essential Teachings of Shakyamuni ...,"Anonymous, Gautama Buddha, Dge-ʼdun-chos-ʼphel",4.26,23550,932,381.0,uary,/work/shelves/154391,NaN,2
134513,443246,Pro M. Caelio Oratio,"Marcus Tullius Cicero, R.G. Austin",3.74,214,16,212.0,-62,/work/shelves/432024,NaN,1
137241,446288,Le gone du Chaâba,Azouz Begag,3.59,748,34,243.0,mber,/work/shelves/674285,NaN,1
140370,449761,The Gospel According to Mark,"Anonymous, Barry Hannah",4.50,551,67,64.0,t 70,/work/shelves/1943661,Bible,2
14697,115505,Anthropology: The Exploration of Human Diversity,Conrad Phillip Kottak,3.85,121,5,720.0,June,/work/shelves/127546,NaN,2
40508,143086,Recogiendo Poemas,Jaime Sabines,4.52,211,13,77.0,Sa,/work/shelves/138021,NaN,2
109367,319588,"Supply Chain Management: Strategy, Planning, a...","Sunil Chopra, Peter Meindl",4.11,508,19,536.0,t 20,/work/shelves/1457919,NaN,2
113781,324553,Fundamentals of Nursing: The Art and Science o...,"Carol R. Taylor, Priscilla LeMone, Carol Lilli...",3.84,182,9,1856.0,uary,/work/shelves/315248,NaN,2
136848,350490,The Virtuous Knight,Margo Maguire,3.73,163,13,304.0,mber,/work/shelves/340739,Medieval Brides,1


In [24]:
info = clean_v4[['Raters', 'Reviewers', 'Pages', 'PublishYear']]

info.head()

,Raters,Reviewers,Pages,PublishYear
0,2469197,40043,652.0,2005
1,2541611,43067,870.0,2003
2,7145889,113905,309.0,1997
3,2766218,53651,352.0,1998
4,2849671,56337,435.0,1999


In [ ]:
labels = clean_v3.review